<a href="https://colab.research.google.com/github/gustavo-moretto/WebScrapping_ImoveisCuritiba/blob/main/webscrapping_imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class AnaliseImoveis:

  def __init__():
    pass

  # instalar dependencias do selenium
  def install_selenium():
    !pip install selenium
    !apt-get update 
    !apt install chromium-chromedriver

  def analise_imoveis( n_paginas,
                    comprar_alugar=['aluguel', 'venda'], 
                    tipos_imoveis = ['apartamento_residencial', 'casa_residencial', 'condominio_residencial', 'cobertura_residencial', 'flat_residencial', 'kitnet_residencial', 'sobrado_residencial']):
  
    lista_bairros = ['Bairr', 'Bairro', 'Bo', 'Bom', 'Boa',  'Camp', 'Campo', 'Campin', 'Campina', 'Campo d', 'Capã', 'Capão',  'Centr',  
                     'Cidad', 'Cidade', 'Crist', 'Cristo', 'Hugo',  'Nov', 'Novo',  'Prado', 'Prado',  'Sant', 'Santo', 'Santa',  'Siti', 
                     'Sitio', 'São', 'Sã',  'Vil', 'Vila', 'Vist', 'Vista',  'Águ', 'Água']
    
    import pandas as pd
    pd.set_option('display.float_format', lambda x: '%.5f' % x)    
    from bs4 import BeautifulSoup
    
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
        
    import seaborn as sns
    import matplotlib.pyplot as plt

    from time import sleep 

    conjunto_imoveis = {}
    c = 0
    first_entry = True   
    for negocio in comprar_alugar:      
      for tipo_imovel in tipos_imoveis:
          driver.get(f"https://www.vivareal.com.br/{negocio}/parana/curitiba/{tipo_imovel}/")
          if first_entry:
            driver.find_element(By.XPATH, '//*[@id="cookie-notifier-cta"]').click()
            first_entry = False          
          print(f'Coletando dados de {negocio} para {tipo_imovel}.' )
          for pagina in range( 1, n_paginas+1 ):                        
            if pagina == 1:
              # extraindo o conteudo da pagina com selenium            
              content = driver.page_source                       
                
              # lê o arquivo html com a biblioteca BeautifulSoup
              soup = BeautifulSoup( content, 'html.parser' )                                              
                
              # seleciona os dados dos imóveis
              # para isso, encontramos todas as tags <a> com a "class" = "property-card__content-link js-card-title"
              # eh a partir daqui que todos os outros dados serao coletados
              imoveis = soup.find_all('a', attrs={'class':"property-card__content-link js-card-title"})

            else:
              try:
                # usando o selenium para navegar para a proxima pagina
                # nesta etapa tive bastante dificuldade, ate perceber que o caminho mais facil seria usar o XPATH do botao "Proxima Pagina"
                # porem, quando o tipo de imovel eh "condominio_residencial", o botao tem outro XPATH
                # criei uma excecao para lidar com isso
                # que a principio funcionou -> pretendo melhorar no futuro
                if tipo_imovel == 'condominio_residencial':
                  try:            
                    driver.find_element(By.XPATH, '//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[4]/button').click()                              
                    sleep(5)
                    content = driver.page_source
                    # lê o arquivo html com a biblioteca BeautifulSoup
                    soup = BeautifulSoup( content, 'html.parser' )               
                    # seleciona os dados dos imóveis
                    # para isso, encontramos todas as tags <a> com a "class" = "property-card__content-link js-card-title"
                    imoveis = soup.find_all('a', attrs={'class':"property-card__content-link js-card-title"})
                  except ElementClickInterceptedException:
                    print('Elemento nao "clicavel"')
                    break
                else:
                  try:           
                    driver.find_element(By.XPATH, '//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/button').click()                              
                    sleep(5)
                    content = driver.page_source
                    # lê o arquivo html com a biblioteca BeautifulSoup
                    soup = BeautifulSoup( content, 'html.parser' )      
                    # seleciona os dados dos imóveis
                    # para isso, encontramos todas as tags <a> com a "class" = "property-card__content-link js-card-title"
                    imoveis = soup.find_all('a', attrs={'class':"property-card__content-link js-card-title"})
                  except ElementClickInterceptedException:
                    break
              except NoSuchElementException:
                print('Botao nao localizado')
                break

            for imovel in range(len(imoveis)):        
                c += 1
                    
                indice_imovel = c
                # extrai a descrição do imóvel
                #descricao = imoveis[imovel].find("span", attrs={"class":"property-card__title js-cardLink js-card-title"}).text                 

                # extrai o endereço e o bairro do imóvel
                endereco = imoveis[imovel].find("span", attrs={"class":"property-card__address"}).text
                end_completo = endereco

                for elemento in range(len(endereco.split())):
                    try:                      
                      if endereco.split()[elemento] == '-':                          
                          end = endereco.split()[elemento + 1]
                          # bairros com nomes compostos -> dessa forma, o nome eh extraido da maneira correta                 
                          if end in lista_bairros:                          
                            endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2]                                                                                    
                          # bairros que possuem tres nomes (p. ex.: Alto da Rua XV)
                          elif end in ['Alto', 'Campina', 'Jardim', 'Campo']:                           
                            try:                              
                              endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2] + ' ' + endereco.split()[elemento + 3]
                            except IndexError:                              
                              endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2]                                                                                     
                              
                          # caso nao haja o nome da rua, o bairro sera igual a 'PR'
                          # para extrair o bairro, devemos extrair o primeiro elemento da lista
                          elif end == 'PR':                          
                            end = endereco.split()[0]
                            # bairros com nomes compostos -> dessa forma, o nome eh extraido da maneira correta                 
                            if end in lista_bairros:                                                    
                              endereco = endereco.split()[0] + ' ' + endereco.split()[1]                                            
                            elif end in ['Alto', 'Campina', 'Jardim', 'Campo']:                                                        
                              try:
                                endereco = endereco.split()[0] + ' ' + endereco.split()[1] + ' ' + endereco.split()[2]
                              except IndexError:
                                endereco = endereco.split()[0] + ' ' + endereco.split()[1]                                                                                                            
                            else:
                              endereco = endereco.split()[0]

                          else:
                            endereco = endereco.split()[elemento + 1]                            
                            break
                    except IndexError:
                      pass                                            
                      break

                # extrai a área do imóvel (m²)
                # como existem imoveis com a area variavel, p. ex. 20-30, criei uma excecao
                # neste caso, a area do imovel sera igual a 0
                # creio que isso traz mais realidade ao dataset tambem
                try:
                  area_m2 = float(imoveis[imovel].find("span", attrs={"class":"property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area"}).text)
                except ValueError:
                  area_m2 = 0

                  # extrai os itens do imóvel, como mobília, churrasqueira, academia...
                  #try:
                      #itens = imoveis[imovel].find("ul", attrs={"class":"property-card__amenities"}).text.split()
                  #except AttributeError:
                      #itens = []
                  #try:
                      #if itens[-1] == '...':
                          #itens = itens[:-1]
                    #except IndexError:
                        #pass

                # número de comodos (caso não possua vaga de garagem, por exemplo, será atribuído valor 0)
                comodos = []
                try:
                    for k in imoveis[imovel].find_all("span", attrs={"class":"property-card__detail-value js-property-card-value"}):
                        comodos.append(float(k.text))
                except ValueError:
                    comodos.append(0)
                if len(comodos) == 3:
                  quartos = float(comodos[0])
                  banheiros = float(comodos[1])
                  vagas = float(comodos[2])
                else:
                  quartos = 0
                  banheiros = 0
                  vagas = 0

                # extrai o valor do aluguel ou da venda
                # devido a forma como os valores eram extraidos, eram retornados diferentes erros
                # precisei criar bastante condicionais
                # o que torna o codigo mais "oneroso" -> pretendo melhorar isso no futuro
                try: 
                  preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text                                                                                                           
                  try:          
                    if (negocio == 'aluguel' and len(preco.split()[1]) >= 5):
                      preco = float(preco.split()[1])      
                      preco = preco * 1000                    
                    elif negocio == 'venda' and len(preco.split()) == 2:
                      preco = float(preco.split()[1])      
                      preco = preco * 1000                   
                    elif negocio == 'venda' and len(preco.split()) == 6:                                  
                      preco = float(preco.split()[1])      
                      preco = preco * 1000
                                         
                    elif negocio == 'venda' and len(preco.split()) == 5:
                      if preco.isnumeric():            
                        preco = float(preco.split()[1])      
                        preco = preco * 1000
                      else:
                        preco = 0                    
                    else:                       
                      preco = preco.split()[1]
                      if negocio == 'aluguel' and len(preco) == 3:                                                
                        preco = preco.replace('.', '')
                        preco = float(preco)      
                        preco = preco * 1000
                      else:           
                        preco = float(preco)      
                        preco = preco * 1000
                  except ValueError:
                    preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text
                    preco = preco.replace('.', '')
                    try:                                
                      preco = float(preco.split()[1])
                    except ValueError:
                      preco = 0
                except AttributeError:                                
                  preco = 0                     

                # extrai o valor do condominio (se não existir, será 0)                
                try:
                    condominio = imoveis[imovel].find("strong", attrs={'class':"js-condo-price"}).text
                    if len(condominio.split()[1]) == 3:
                      condominio = condominio.replace('.', '')
                      condominio = float(condominio.split()[1])
                    else:
                      condominio = condominio.replace('.', '')                      
                      condominio = float(condominio.split()[1])
                                   
                except AttributeError or ValueError:
                    condominio = 0
                    

                if endereco is not None and endereco[-1] == ',':
                  endereco = endereco[:-1]
                
                conjunto_imoveis[c] = (negocio, tipo_imovel, indice_imovel, end_completo, endereco, area_m2, quartos, banheiros, vagas, preco, condominio)                
                
    return conjunto_imoveis 

  # criar base de dados com o conjunto de imoveis coletados (salvar em arquivo csv)
  def base_dados( conjunto_imoveis ):

    import pandas as pd

    from google.colab import files
    from datetime import date

    # criacao do dataframe
    df = pd.DataFrame.from_dict(conjunto_imoveis)
    df = df.T
    df = df.rename(columns={ 0:'comprar_alugar', 1:'tipo_imovel', 2:'indice', 3:'end_completo', 4:'bairro', 5:'area_m2', 6:'quartos', 7:'banheiros', 8:'vagas_garagem', 9:'preco', 10:'condominio'})
    
    colunas_float = ['area_m2', 'quartos', 'banheiros', 'vagas_garagem', 'preco', 'condominio']
    for coluna in colunas_float: 
      df[coluna] = df[coluna].astype( float )
        
    print(f'Base de dados criada: {len(df)} imoveis cadastrados.')

    # download da base de dados
    download_basedados = str(input('Deseja fazer o download da base de dados [s/n]: '))
    if download_basedados.upper() == 'S':
      today = date.today()
      print(f'Download do arquivo df_imoveis_{today}.csv')
      df.to_csv(f'df_imoveis_{today}.csv')
      files.download(f'df_imoveis_{today}.csv')
    else:
      print('Base de dados nao foi baixada.')

    # arrumando o nome de alguns bairros
    df['bairro'] = df['bairro'].apply( lambda x: 'Jardim Botânico' if x == 'Jardim Botânico, Curitiba' else x )
    df['bairro'] = df['bairro'].apply( lambda x: 'Alto Boqueirão' if x == 'Alto Boqueirão, Curitiba' else x )
    df['bairro'] = df['bairro'].apply( lambda x: 'Alto da Rua XV' if x == 'Alto da Rua' else x )
    df['bairro'] = df['bairro'].apply( lambda x: 'Jardim Social' if x == 'Jardim Social, Curitiba' else x )
    df['bairro'] = df['bairro'].apply( lambda x: 'Capão da Imbuia' if x == 'Capão da' else x )
    df['bairro'] = df['bairro'].apply( lambda x: 'Campina do Siqueira' if x == 'Campina do' else x )
    df['bairro'] = df['bairro'].apply( lambda x: 'Campo de Santana' if x == 'Campo de' else x )

    return df